# Tearing Avoidance Policy Analysis

In [1]:
import os, sys
import numpy as np
data_dir = os.path.join('..', 'data')
tm_avoidance_dir = os.path.join('..', 'tm_avoidance_model')

# load simulator
sys.path.insert(0, '..')
from tm_avoidance_model import Env

### DIII-D Data

- x0: **0D Parameters**. This includes magnetic signals and actuators. There are 11 in total.
- x1: **1D Plasma Profiles**: One-dimensional kinetic and magnetic profiles mapped in a magnetic flux coordinate. The profiles aree electron density, electron temperature, ion rotation, safety factor, and plasma pressure. v Ie 5 plasma profiles mapped on 33 grids. 


In [2]:
# x0 metrics
x0_mean_path = os.path.join(tm_avoidance_dir, 'x0_mean.npy')
x0_mean = np.load(x0_mean_path)
x0_std_path = os.path.join(tm_avoidance_dir, 'x0_std.npy')
x0_std = np.load(x0_std_path)
print("x0 mean:", x0_mean.shape)
# x1 metrics
x1_mean_path = os.path.join(tm_avoidance_dir, 'x1_mean.npy')
x1_mean = np.load(x1_mean_path)
x1_std_path = os.path.join(tm_avoidance_dir, 'x1_std.npy')
x1_std = np.load(x1_std_path)
print("x1 mean:", x1_mean.shape)


x0 mean: (11,)
x1 mean: (33, 5)


### Observation/State Space

The observation is a 33x5 plasma profile which can take values in the range [-2, 2].

The 5 values represent ne, te, 1/q, pres, rot.

- **ne**: electron density
- **te**: electron temperature
- **$\frac{1}{q}$**: one divided by the safety factor (q)
- **pres**: plasma pressure 
- **rot**: ion rotation
```python
self.observation_space = gym.spaces.Box(
    low = -2 * np.ones_like(self.x1_mean), # Shape: (33, 5)
    high = 2 * np.ones_like(self.x1_mean), # Shape: (33, 5)
    dtype = np.float32
)
```

The state is also size 33x5 and is generated by picking a (33x5) element from one of the N samples in the dataset.
```python
self.idx = np.random.randint(len(self.x0))
(self.x1[self.idx] - s_mean) / s_std
```
where $s_{mean}$ and $s_{std}$ are the mean and standard deviation of the state (ne, te, 1/q, pres, rot): 
```python
s_mean = np.array([3.977, 1.587, 0.5103, 23303., 42.93])
s_std = np.array([2.764, 1.560, 0.4220, 35931., 58.47])
```

### Action Space

The actions are the **total beam power and plasma top triangularity**. Other actuators on the tokamak are assumed to be fixed. The plasma control system (PCS) holds these other actuators such as beam torque, plasma current, bottom triangularity and plasma elongation. These are fixed in order to maintain a $q_{95}\approx 3$ for ITER baseline conditions.

I still am uncertain why the code has 3 actions when the paper mentions only 2. The third action is bottom triangularity which the authors say are fixed in their nature paper. 

```python
a_mean = np.array([4072.8761393229165, 0.6, 0.41])
a_std = np.array([3145.5935872395835, 0.31, 0.31])

```
Actions correspond to beam power ('pinj'), top triangularity ('tritop_EFIT01'), and bottom triangularity ('tribot_EFIT01')



### Dynamic (Reward) Model

The reward model is defined mostly by the dynamic model, a supervised DNN that predicts $\beta_n$ and tearability.

The dynamic model is NOT a simulator that predicts the next state given a current state and action.

Below is pseudocode giving the general idea of what the model is doing.

#### labels for inputs and outputs of reward model
```python
# at t+dt (pinj and tinj both define injected power and injected torque)
inputs_0d = ['bt', 'ip', 'pinj', 'tinj', 'R0_EFITRT1', 'kappa_EFITRT1', 'tritop_EFIT01', 'tribot_EFIT01', 'gapin_EFIT01', 'ech_pwr_total', 'EC.RHO_ECH']
# at t
inputs_1d = ['thomson_density_mtanh_1d', 'thomson_temp_mtanh_1d', '1/qpsi_EFITRT1', 'pres_EFIT01', 'cer_rot_csaps_1d']
# at t+dt
outputs = ['betan_EFITRT1', 'tm_label']
```

```!Python
define state, action
# Define tearability threshold
threshold = 0.5
# Predict next step
y = dynamic_model.predict(state, action)
betan, tearability = y[0][0], sigmoid(y[1][0])
# Estimate reward
if tearability < threshold:
    reward = betan
else:
    reward = threshold - tearability

```

This is a KNOWN model (trained neural network) and is only used to determine the reward for the given (s, a) pair.

### environment: step function

The step() function should transition to a new state based on an action, but it does not.
Instead, the observation is **randomly sampled from experimental data**.

```python

# index is randomly sampled in the reset function after each step because step returns done=True   
self.idx = np.random.randint(len(self.x0))

def step(self, action):
    # Take action
    action1 = np.clip(action * a_std + a_mean, clip_action1, clip_action2)
    x0_tmp, x1_tmp = self.x0[[self.idx]].copy(), self.x1[[self.idx]].copy()
    x0_tmp[0, 2] = action1[0]
    x0_tmp[0, 3] = min(1.0, action1[0] * self.x0_mean[3] / self.x0_mean[2])
    x0_tmp[0, 6] = action1[1]
    x0_tmp[0, 7] = action1[2]

    # Predict next step
    y = self.predictors[self.i_model].predict([x0_tmp, x1_tmp])
    betan, tearability = y[0][0], sigmoid(y[1][0])

    # Estimate reward
    if tearability < threshold:
        reward = betan
    else:
        reward = threshold - tearability
    print(self.episodes, action[0], betan, tearability, reward)
    return (self.x1[self.idx] - s_mean) / s_std, reward, True, {}
```

#### Simulate Original DIII-D data

In [3]:
def simulate_tokamak_data(n, mu_0, sigma_0, mu_1, sigma_1):
    '''
    Simulate DIII-D data based upon calculated mean and standard deviations of x0 and x1 from Seo et al.
    Parameters:
        n (int):
            total number of samples
        mu_0 (np.array):
            mean of 0D parameters from dataset. Shape = (11,)
        sigma_0 (np.array):
            standard deviation of 0D parameters from dataset. Shape = (11,)
        mu_1 (np.array):
            mean of plasma profile from dataset. Shape = (33, 5)
        sigma_1 (np.array):
            standard deviation of 0D parameters from dataset. Shape = (33, 5)
    '''
    x0 = np.random.normal(loc=mu_0, scale=sigma_0, size=(n, mu_0.shape[0]))
    x1 = np.random.normal(loc=mu_1, scale=sigma_1, size=(n, mu_1.shape[0], mu_1.shape[1]))
    return x0, x1

In [4]:
N = 10000
simulated_x0, simulated_x1 = simulate_tokamak_data(N, x0_mean, x0_std, x1_mean, x1_std)
print('0D Parameters:', simulated_x0.shape)

print('1D Plasma Profile:', simulated_x1.shape)

0D Parameters: (10000, 11)
1D Plasma Profile: (10000, 33, 5)


### Load environment with simulated data

In [5]:
env = Env(x0_data=simulated_x0, x1_data=simulated_x1)
s = env.reset()

313/313 [==============================] - 2s 4ms/step


In [6]:
a = np.array([0.5, 0.5, 0.5])
s_prime, reward, done, info = env.step(a)
print("reward", reward)
print("Episode Terminated:", done)
print("Testing if step function is deterministic given state and action")
s_prime, reward, done, info = env.step(a)
print("reward", reward)
print("Episode Terminated:", done)

1/1 [==============================] - 0s 42ms/step
reward -0.4615461693020966
Episode Terminated: True
Testing if step function is deterministic given state and action
1/1 [==============================] - 0s 47ms/step
reward -0.4615461693020966
Episode Terminated: True


### Load tearability predictor (reward model)

This is what is happening under the hood during an episode. An episode in this work is defined as a single step because random samples are drawn from the dataset to get to the next state (independent of current state and action).

In [7]:
from keras import models
import math

def f1_m(y_true, y_pred):
    '''
    Requires function for f1 score to load model. Only used for the dynamic model training.
    '''
    return 1.0

def sigmoid(z):
    return 1/(1+math.e**(-z))

# get random sample from generated dataset
idx = np.random.randint(len(simulated_x0))
predictor_dir = os.path.abspath(os.path.join('..', 'tm_prediction_model'))
threshold = 0.5
#  Load model from folder with TensorFlow's Protocol Buffer files
predictors = [models.load_model(os.path.join(predictor_dir, f'best_model_{i}'), custom_objects={'f1_m':f1_m}) for i in range(5)]
y = np.mean([p.predict([simulated_x0[[idx]], simulated_x1[[idx]]]) for p in predictors], axis=0)
beta_n, tearability = y[0][0], sigmoid(y[1][0])

1/1 [==============================] - 0s 399ms/step


In [8]:
print("beta_n:", beta_n)
print("tearability:", tearability)

# Estimate reward
if tearability < threshold:
    reward = beta_n
else:
    reward = threshold - tearability

print("reward:", reward)

beta_n: 1.3993876
tearability: 0.0013292713030323977
reward: 1.3993876
